# 🛰️ Sentinel-2 Data Analysis & Vegetation Monitoring (NDVI)

This notebook demonstrates how to:
1. Define an Area of Interest (AOI) on a map.
2. Search for Sentinel-2 satellite imagery using `sentinelsat`.
3. Download necessary spectral bands (Red/B4 and NIR/B8).
4. Calculate and visualize the **Normalized Difference Vegetation Index (NDVI)** to assess crop health.

### Pre-requisites:
- A **Copernicus Open Access Hub** account (username/password).
- Installed dependencies (`geopandas`, `sentinelsat`, `folium`, `rasterio`).

In [4]:
import os
import folium
import geopandas as gpd
from pystac_client import Client
from datetime import date

# 🔐 Set your Copernicus Data Space Ecosystem credentials (CDSE)
# Use the STAC API instead of the legacy Sentinel API (SciHub)
COPERNICUS_USER = os.getenv('COPERNICUS_USER', 'urania.giannopoulou@gmail.com')
COPERNICUS_PASS = os.getenv('COPERNICUS_PASS', 'TzL#us7A+,*q)+^')

# Connect to the CDSE STAC Catalog
# Documentation: https://documentation.dataspace.copernicus.eu/APIs/STAC.html
catalog_url = "https://catalogue.dataspace.copernicus.eu/stac"
client = Client.open(catalog_url)

## 1. Define Area of Interest (AOI)
Create a map to visualize the location. We'll simulate an agricultural area (e.g., in Italy or Greece).

In [5]:
m = folium.Map([41.9028, 12.4964], zoom_start=10) # Centered on Rome

# Define a simple GeoJSON for the query (or load from file)
aoi_geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [12.4, 41.8],
            [12.6, 41.8],
            [12.6, 42.0],
            [12.4, 42.0],
            [12.4, 41.8]
          ]
        ]
      }
    }
  ]
}

folium.GeoJson(aoi_geojson, name="AOI").add_to(m)
m

## 2. Search for Sentinel-2 Imagery
We use the AOI geometry and a date range to find suitable images (low cloud cover).

In [6]:
# Use STAC Search API to find Sentinel-2 L2A (Level 2A) imagery
# "SENTINEL-2" collection ID in CDSE
collection = "SENTINEL-2" 

# Define search parameters
search = client.search(
    collections=[collection],
    intersects=aoi_geojson['features'][0]['geometry'],
    datetime="2023-06-01/2023-06-30",
    query={"eo:cloud_cover": {"lt": 20}},
    max_items=100
)

items = search.item_collection()
print(f"Found {len(items)} products.")

# Convert to minimal metadata list for display
products_list = []
for item in items:
    products_list.append({
        'id': item.id,
        'date': item.datetime.isoformat(),
        'cloud_cover': item.properties['eo:cloud_cover'],
        'platform': item.properties['platform']
    })

# Display first 5 results
import pandas as pd
products_df = pd.DataFrame(products_list)
products_df.head()

Found 0 products.


""


## 3. Calculate NDVI
The formula for NDVI is:
$$ NDVI = \frac{(NIR - Red)}{(NIR + Red)} $$
- **NIR (Near-Infrared)**: Band 8 in Sentinel-2
- **Red**: Band 4 in Sentinel-2

Higher NDVI values (closer to 1.0) indicate healthy, green vegetation.

In [ ]:
# Sort by least cloud cover and pick the best image
best_item = items[0] # Default to first
min_cloud = 100

for item in items:
    if item.properties['eo:cloud_cover'] < min_cloud:
        min_cloud = item.properties['eo:cloud_cover']
        best_item = item

print(f"Selected Image ID: {best_item.id}")
print(f"Cloud Cover: {min_cloud}%")
print(f"Date: {best_item.datetime.date()}")

# List available assets (bands)
print("Available assets:", list(best_item.assets.keys()))

In [ ]:
import rasterio
from rasterio.features import geometry_mask
from rasterio.warp import transform_geom
from rasterio.windows import from_bounds
import matplotlib.pyplot as plt
import numpy as np

# Sentinel-2 Bands: 'red' -> B04, 'nir' -> B08

# Get the URLs for the bands
red_href = best_item.assets['red'].href
nir_href = best_item.assets['nir'].href
print(f"Red Band URL: {red_href}")

# In CDSE, accessing the assets often requires authentication even for public data via S3 or signed URLs.
# If running locally without specific token setup, we might need a workaround using vsicurl with authentication.
# For this example, we will assume standard HTTPS access. If 403, we need to sign the request.

# Function to read a specific window (AOI)
def read_band_aoi(href, aoi_geometry):
    # Configure GDAL to use the VSI curl system which handles HTTP ranges
    with rasterio.Env(GDAL_HTTP_COOKIEFILE='cookies.txt', GDAL_HTTP_COOKIEJAR='cookies.txt'):
        with rasterio.open(href) as src:
            aoi_crs = 'EPSG:4326'
            img_crs = src.crs

            # Transform AOI to image CRS
            aoi_projected = transform_geom(aoi_crs, img_crs, aoi_geometry)

            # Calculate the window to read
            bounds = rasterio.features.bounds(aoi_projected)
            window = from_bounds(*bounds, transform=src.transform)

            # Read the data in that window
            return src.read(1, window=window)

try:
    # Read the bands
    print("Attempting to download Red band window...")
    red_data = read_band_aoi(red_href, aoi_geojson['features'][0]['geometry'])
    print("Attempting to download NIR band window...")
    nir_data = read_band_aoi(nir_href, aoi_geojson['features'][0]['geometry'])
    
    print(f"Data shapes: Red {red_data.shape}, NIR {nir_data.shape}")

    # Simple visualization of the Red band
    plt.figure(figsize=(6,6))
    plt.imshow(red_data, cmap='Reds')
    plt.title("Red Band (Raw Data)")
    plt.colorbar()
    plt.show()

except Exception as e:
    print(f"Error reading data: {e}")
    print("POSSIBLE FIX: The new Copernicus Ecosystem often requires S3 signed URLs.")
    print("If this fails, we can switch to using the 'stackstac' library which handles this easier for xarray.")

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Sentinel-2 Bands:
# 'red' -> B04 (10m resolution)
# 'nir' -> B08 (10m resolution)

# Get the URLs for the bands (Note: CDSE URLs might require token handling or proper S3 access)
# For simplicity with CDSE, we often use the href directly if authenticated or public.
red_href = best_item.assets['red'].href
nir_href = best_item.assets['nir'].href

print(f"Red Band URL: {red_href}")
print(f"NIR Band URL: {nir_href}")

# Function to read a specific window (AOI) from the remote GeoTIFF
# This avoids downloading the whole 800MB file!
from rasterio.features import geometry_mask

def read_band_aoi(href, aoi_geometry):
    with rasterio.open(href) as src:
        # Create a window from the AOI geometry
        # We need to project the AOI to the image CRS first!
        from rasterio.warp import transform_geom
        
        aoi_crs = 'EPSG:4326'  # Our GeoJSON is simpler lat/lon
        img_crs = src.crs
        
        # Transform AOI to image CRS
        aoi_projected = transform_geom(aoi_crs, img_crs, aoi_geometry)
        
        # Calculate the window to read
        from rasterio.windows import from_bounds
        bounds = rasterio.features.bounds(aoi_projected)
        window = from_bounds(*bounds, transform=src.transform)
        
        # Read the data in that window
        data = src.read(1, window=window)
        
        # Return data and the transform for that specific window
        return data, src.window_transform(window)

try:
    # Read the bands (Red and NIR)
    red_data, transform = read_band_aoi(red_href, aoi_geojson['features'][0]['geometry'])
    nir_data, _ = read_band_aoi(nir_href, aoi_geojson['features'][0]['geometry'])
    
    print(f"Data shapes: Red {red_data.shape}, NIR {nir_data.shape}")
    
    # Simple visualization of the Red band
    plt.figure(figsize=(6,6))
    plt.imshow(red_data, cmap='Reds')
    plt.title("Red Band (Raw Data)")
    plt.colorbar()
    plt.show()

except Exception as e:
    print(f"Error reading data: {e}")
    print("If you get a 403 Forbidden, we may need to configure the CDSE S3 credentials or token.")